In [15]:
import pandas as pd

df = pd.read_csv('./weiss-contributions-2017.csv')

In [16]:
df.columns

Index(['CO_ID', 'ContributionAmount', 'ContributionDate', 'LastName',
       'FirstName', 'MI', 'Suffix', 'Address1', 'Address2', 'City', 'State',
       'Zip', 'Explanation', 'RecordID', 'FiledDate', 'ContributionType',
       '      "ReceiptType"', 'ContributorType', 'Electioneering',
       'CommitteeType', 'CommitteeName', 'CandidateName', 'Employer',
       'Occupation', 'Amended', 'Amendment', 'AmendedRecordID', 'Jurisdiction',
       'OccupationComments'],
      dtype='object')

In [17]:
pd.set_option('display.max_columns', 50)
df.head()

,CO_ID,ContributionAmount,ContributionDate,LastName,FirstName,MI,Suffix,Address1,Address2,City,State,Zip,Explanation,RecordID,FiledDate,ContributionType,"""ReceiptType""",ContributorType,Electioneering,CommitteeType,CommitteeName,CandidateName,Employer,Occupation,Amended,Amendment,AmendedRecordID,Jurisdiction,OccupationComments
0,20175032081,80.0,2017-05-08 00:00:00,WEISER,PHILIP,J,NaN,200 S. ULSTER ST,NaN,DENVER,CO,80230.0,P.O. BOX PAYMENT,4554220,2017-07-17 00:00:00,Non-Monetary (Itemized),In-Kind,Candidate,NaN,Candidate Committee,PHIL WEISER FOR COLORADO,PHILIP JACOB WEISER,NaN,NaN,N,N,0,STATEWIDE,NaN
1,20175032081,5.0,2017-05-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4554088,2017-07-17 00:00:00,Monetary (Non-Itemized),Other,NaN,NaN,Candidate Committee,PHIL WEISER FOR COLORADO,PHILIP JACOB WEISER,NaN,NaN,N,N,0,STATEWIDE,NaN
2,20175032081,10.0,2017-05-10 00:00:00,SOOTER,MONTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4554089,2017-07-17 00:00:00,Monetary (Itemized),Credit/Debit Card,Individual,NaN,Candidate Committee,PHIL WEISER FOR COLORADO,PHILIP JACOB WEISER,NaN,NaN,N,N,0,STATEWIDE,NaN
3,20175032081,20.0,2017-05-10 00:00:00,SOOTER,MONTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4554090,2017-07-17 00:00:00,Monetary (Itemized),Credit/Debit Card,Individual,NaN,Candidate Committee,PHIL WEISER FOR COLORADO,PHILIP JACOB WEISER,NaN,NaN,N,N,0,STATEWIDE,NaN
4,20175032081,5.0,2017-05-10 00:00:00,UNGEMAH,DAVID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4554149,2017-07-17 00:00:00,Monetary (Itemized),Credit/Debit Card,Individual,NaN,Candidate Committee,PHIL WEISER FOR COLORADO,PHILIP JACOB WEISER,NaN,NaN,N,N,0,STATEWIDE,NaN


In [29]:
df['person'] = df['FirstName'] + ' ' + df['LastName'] + ' ' + df['Address1'] + ' ' + df['City']

In [26]:
print('''There are {} rows ({} unique people). {} of them have an Occupation, {} have OccupationComments'''.format(
        df.shape[0],
        df.person.nunique(),
        df.Occupation.count(),
        df.OccupationComments.count()))

There are 3200 rows (2609 unique people). 2543 of them have an Occupation, 2 have OccupationComments


In [100]:
contribs_by_person = df.groupby('person').ContributionAmount.sum()

In [108]:
df[df.LastName != 'COLORADO DEMOCRATIC PARTY'].groupby('colorado').mean()

,CO_ID,ContributionAmount,Zip,RecordID,Electioneering,AmendedRecordID
colorado,,,,,,
False,2.017503e+10,355.351843,31178.686528,4.618901e+06,NaN,20547.844944
True,2.017503e+10,309.071620,80334.064963,4.640702e+06,NaN,38125.109571


In [51]:
df['colorado'] = df.State == 'CO'

In [97]:
df.ContributionAmount.mean()

326.534221875

In [98]:
df.groupby('colorado').ContributionAmount.mean()

colorado
False    355.351843
True     315.431329
Name: ContributionAmount, dtype: float64

In [87]:
asdf = df.groupby(['Occupation', 'colorado']).CO_ID.count().unstack().reset_index()
asdf['percent_out_of_state'] = asdf[False].fillna(0) / (asdf[False].fillna(0) + asdf[True])
asdf['n'] = asdf[False].fillna(0) + asdf[True]
asdf.drop([False, True], axis=1, inplace=True)

In [95]:
asdf.sort_values(by='percent_out_of_state', ascending=False).reset_index(drop=True)

colorado,Occupation,percent_out_of_state,n
0,Lobbyist,0.818182,11.0
1,Insurance Industry,0.666667,6.0
2,Teacher/Education,0.503106,161.0
3,Media/Public Relations,0.466667,60.0
4,Government/Civil,0.444444,36.0
5,Consultant,0.348624,109.0
6,Attorney/Legal,0.330806,1055.0
7,Homemaker,0.290323,31.0
8,Other,0.274854,171.0
9,Environmental Services,0.250000,4.0


In [ ]:
asdf.sort_values(by='percent_out_of_state', ascending=False).reset_index

In [ ]:
df.groupby(['colorado', 'Occupation']).CO_ID.count().sort_values

In [ ]:
df.groupby(['colorado', 'Occupation']).CO_ID.count().sort_values

In [52]:
df.groupby('colorado').ContributionAmount.mean()

colorado
False    355.351843
True     315.431329
Name: ContributionAmount, dtype: float64

In [45]:
df[df.person.isin(contribs_by_person[contribs_by_person == 1150].index)].groupby('person').first().Occupation.value_counts()

Attorney/Legal                           193
Teacher/Education                         32
Financial/Investment                      28
Retired                                   28
Other                                     20
Science/Technology                        19
Healthcare/Medical                        14
Media/Public Relations                    14
Real Estate Professional                  12
Not Currently Employed                    11
Consultant                                 9
General Business                           5
Manufacturing                              5
Homemaker                                  5
Mining/Energy                              4
Construction/Engineering                   4
Government/Civil                           4
Retail Sales                               3
Lobbyist                                   3
Office and Administrative Occupations      2
Student                                    2
Food Services                              1
Agricultur

In [55]:
(contribs_by_person == 1150).sum()

420

In [20]:
df.Occupation.value_counts()

Attorney/Legal                           1055
Retired                                   233
Other                                     171
Teacher/Education                         161
Healthcare/Medical                        132
Financial/Investment                      129
Consultant                                109
Science/Technology                        105
General Business                           76
Real Estate Professional                   68
Not Currently Employed                     66
Media/Public Relations                     60
Government/Civil                           36
Homemaker                                  31
Construction/Engineering                   20
Mining/Energy                              15
Retail Sales                               14
Manufacturing                              12
Office and Administrative Occupations      11
Lobbyist                                   11
Food Services                               8
Agriculture                       

In [109]:
df.groupby("Occupation").ContributionAmount.agg({'mean': 'mean', 'n': 'count', 'sum': 'sum'}).sort_values(by='sum', ascending=False)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if __name__ == '__main__':


,n,sum,mean
Occupation,,,
Attorney/Legal,1055,437967.92,415.135469
Retired,233,79372.70,340.655365
Teacher/Education,161,65212.48,405.046460
Other,171,61306.10,358.515205
Financial/Investment,129,60919.00,472.240310
Science/Technology,105,45493.50,433.271429
Healthcare/Medical,132,41306.44,312.927576
Consultant,109,32509.75,298.254587
Real Estate Professional,68,28080.97,412.955441
